# Análise de Churn

### Definição do problema

A retenção de clientes é um dos desafios mais críticos para empresas que operam em mercados competitivos. O churn, ou taxa de cancelamento de clientes, representa a proporção de clientes que deixam de utilizar um serviço dentro de um determinado período. Compreender os fatores que influenciam o churn é essencial para que as empresas desenvolvam estratégias eficazes de retenção, reduzindo perdas e maximizando a receita.

Neste projeto, realizaremos uma análise detalhada do comportamento dos clientes com base em um conjunto de dados contendo informações como idade, tempo de contrato, frequência de uso, total gasto, interações com o call center e outras variáveis relevantes. O objetivo é identificar padrões que diferenciam clientes que permanecem dos que cancelam, permitindo a criação de estratégias para minimizar o churn.

Por meio de estatísticas descritivas, visualizações de dados e modelagem preditiva, buscamos responder às seguintes perguntas-chave:
- Quais características dos clientes estão mais associadas ao churn?
- Clientes com maior tempo de contrato têm menor probabilidade de cancelar?
- Qual o impacto das interações com o call center na retenção?
- O valor total gasto influencia a decisão de cancelar o serviço?
- Ao final da análise, apresentaremos insights e recomendações práticas para reduzir o churn e melhorar a experiência do cliente.

## Dicionário de dados

**Dicionário de dados** para a sua tabela:  

| Nome da Variável            | Tipo       | Descrição |
|-----------------------------|-----------|-----------|
| **CustomerID**              | Inteiro    | Identificação única do cliente. |
| **idade**                   | Inteiro    | Idade do cliente (em anos). |
| **sexo**                    | Categórico | Gênero do cliente (ex.: "M" para masculino, "F" para feminino). |
| **tempo_como_cliente**       | Inteiro    | Tempo total (em meses) que o cliente está com a empresa. |
| **frequencia_uso**           | Numérico   | Frequência média de uso do serviço pelo cliente. |
| **ligacoes_callcenter**      | Inteiro    | Número de ligações realizadas para o call center. |
| **dias_atraso**             | Inteiro    | Número de dias que o cliente já atrasou pagamentos. |
| **assinatura**               | Categórico | Tipo de plano/assinatura do cliente (ex.: "Pré-pago", "Pós-pago", "Controle"). |
| **duracao_contrato**         | Inteiro    | Duração total do contrato do cliente (em meses). |
| **total_gasto**              | Numérico   | Valor total gasto pelo cliente durante o período com a empresa. |
| **meses_ultima_interacao**   | Inteiro    | Tempo (em meses) desde a última interação do cliente com a empresa. |
| **cancelou**                 | Binário    | Indica se o cliente cancelou o serviço (1 = Sim, 0 = Não). |


### 2. Exploração de dadoss

- Analise a distribuição das variáveis:
    - Idade dos clientes
    - Frequência de uso
    - Total gasto
    - Tempo como cliente
- Verifique possíveis outliers que possam distorcer a análise.

In [32]:
# importando a base dados
import pandas as pd
import plotly.express as px


In [33]:
# carregando os dados
tabela = pd.read_csv(r'C:\projetos\analise-churn-empresa-telefonia\dados\cancelamentos.csv')

In [34]:
tabela.head()

,CustomerID,idade,sexo,tempo_como_cliente,frequencia_uso,ligacoes_callcenter,dias_atraso,assinatura,duracao_contrato,total_gasto,meses_ultima_interacao,cancelou
0,2.0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.0,17.0,1.0
1,3.0,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.0,6.0,1.0
2,4.0,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.0,3.0,1.0
3,5.0,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.0,29.0,1.0
4,6.0,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.0,20.0,1.0


In [35]:
# verificando quantos registros e variáveis
tabela.shape

(881666, 12)

In [36]:
# nome das colunas
tabela.columns

Index(['CustomerID', 'idade', 'sexo', 'tempo_como_cliente', 'frequencia_uso',
       'ligacoes_callcenter', 'dias_atraso', 'assinatura', 'duracao_contrato',
       'total_gasto', 'meses_ultima_interacao', 'cancelou'],
      dtype='object')

In [37]:
# informações
tabela.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 881666 entries, 0 to 881665
Data columns (total 12 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   CustomerID              881664 non-null  float64
 1   idade                   881664 non-null  float64
 2   sexo                    881664 non-null  object 
 3   tempo_como_cliente      881663 non-null  float64
 4   frequencia_uso          881663 non-null  float64
 5   ligacoes_callcenter     881664 non-null  float64
 6   dias_atraso             881664 non-null  float64
 7   assinatura              881661 non-null  object 
 8   duracao_contrato        881663 non-null  object 
 9   total_gasto             881664 non-null  float64
 10  meses_ultima_interacao  881664 non-null  float64
 11  cancelou                881664 non-null  float64
dtypes: float64(9), object(3)
memory usage: 80.7+ MB


In [38]:
# verificando se existem dados nulos
tabela.isnull().sum()

CustomerID                2
idade                     2
sexo                      2
tempo_como_cliente        3
frequencia_uso            3
ligacoes_callcenter       2
dias_atraso               2
assinatura                5
duracao_contrato          3
total_gasto               2
meses_ultima_interacao    2
cancelou                  2
dtype: int64

In [39]:
# removendo os valores vazios
tabela = tabela.dropna()

In [40]:
# conferindo a  nova quantidade de registros
tabela.shape

(881659, 12)

In [41]:
# conferindo se os valores nulos foram removidos
tabela.isnull().sum()

CustomerID                0
idade                     0
sexo                      0
tempo_como_cliente        0
frequencia_uso            0
ligacoes_callcenter       0
dias_atraso               0
assinatura                0
duracao_contrato          0
total_gasto               0
meses_ultima_interacao    0
cancelou                  0
dtype: int64

In [42]:
# removendo a coluna CustomerID
tabela.drop(columns='CustomerID', axis=1, inplace=True)

In [43]:
# calculando a quantidade de cancelamentos 
print(tabela['cancelou'].value_counts())
print(tabela['cancelou'].value_counts(normalize=True).map("{:.2%}".format))


cancelou
1.0    499993
0.0    381666
Name: count, dtype: int64
cancelou
1.0    56.71%
0.0    43.29%
Name: proportion, dtype: object


In [44]:
# # Vamos através de graficos verificar a relação entre as variáveis e o cancelamento
# for variavel in tabela.columns:
#     print(f'{variavel.upper()}:')
#     #display(tabela.groupby(variavel).mean(numeric_only=True))
#     grafico = px.histogram(tabela, x=variavel, color='cancelou', color_discrete_map={1.0: "red", 0.0:"blue" })
#     grafico.show()
#     print(100*'--------')

## 3. Análise Univariada
- Estatísticas descritivas para entender cada variável.
- Histogramas e boxplots para variáveis numéricas.
- Contagem de categorias para variáveis categóricas (sexo, assinatura, cancelou).

#### Cenário 1: Excluir todos os planos Mensais

In [45]:
## excluindo os clientes do plano mensal para verificar como ficaria a distribuição dos cancelamentos
tabela_filtro_assinatura = tabela.loc[tabela['duracao_contrato'] !='Monthly',:]
tabela_filtro_assinatura['cancelou'].value_counts()
tabela_filtro_assinatura['cancelou'].value_counts(normalize=True)

cancelou
0.0    0.539492
1.0    0.460508
Name: proportion, dtype: float64

#### Cenário 2: Excluir todos os clientes que ligaram mais de 5 vezes

In [46]:
# excluindo os clientes que ligaram mais de 5 vezes
tabela_filtro_ligacoes = tabela.loc[tabela['ligacoes_callcenter'] <= 5,:]
tabela_filtro_ligacoes['cancelou'].value_counts()
tabela_filtro_ligacoes['cancelou'].value_counts(normalize=True)

cancelou
0.0    0.592066
1.0    0.407934
Name: proportion, dtype: float64

#### Cenário 3 : Excluir todos os clientes que atrasaram o pagamento mais de 20 dias

In [47]:
# excluindo os clientes que atrasaram mais de 20 dias
tabela_filtro_dias_atraso = tabela.loc[tabela['dias_atraso'] <= 20,:]
tabela_filtro_dias_atraso['cancelou'].value_counts()
tabela_filtro_dias_atraso['cancelou'].value_counts(normalize=True)

cancelou
0.0    0.534845
1.0    0.465155
Name: proportion, dtype: float64

#### Cenário 4: Excluir todos os clientes acima de 50 anos

In [48]:
# excluindo os clientes acima de 50 anos
tabela_filtro_idade = tabela.loc[tabela['idade'] <= 50,:]
tabela_filtro_idade['cancelou'].value_counts()
tabela_filtro_idade['cancelou'].value_counts(normalize=True)

cancelou
0.0    0.531308
1.0    0.468692
Name: proportion, dtype: float64

## 4. Análise Bivariada (Churn vs Outras Variáveis)
- Compare as características dos clientes que cancelaram (cancelou = 1) com os que não cancelaram (cancelou = 0).
- Idade média dos que cancelam vs. os que permanecem.
- Relação entre total gasto e cancelamento.
- Clientes que mais ligam para o call center têm maior taxa de churn?
- Clientes com mais tempo de contrato têm menor chance de cancelar?

#### Cenário 6: Excluir todos os clientes do plano mensal e que ligaram mais de 5 vezes 

In [49]:
## excluindo os clientes do plano mensal para verificar como ficaria a distribuição dos cancelamentos
tabela_filtro_assinatura_e_ligacaoes = tabela.loc[(tabela['duracao_contrato'] !='Monthly') & (tabela['ligacoes_callcenter'] <= 5),:]
tabela_filtro_assinatura_e_ligacaoes['cancelou'].value_counts()
tabela_filtro_assinatura_e_ligacaoes['cancelou'].value_counts(normalize=True)

cancelou
0.0    0.694777
1.0    0.305223
Name: proportion, dtype: float64

#### Cenário 7: Excluir todos os clientes do plano mensal e que atrasaram o pagamento mais de 20 dias

In [50]:
#### excluindo os clientes do plano mensal e que atrasaram o pagamento mais de 20 dias
tabela_filtro_assinatura_e_dias_atraso = tabela.loc[(tabela['duracao_contrato'] !='Monthly') & (tabela['dias_atraso'] <= 20),:]
tabela_filtro_assinatura_e_ligacaoes['cancelou'].value_counts()
tabela_filtro_assinatura_e_dias_atraso['cancelou'].value_counts(normalize=True)

cancelou
0.0    0.640875
1.0    0.359125
Name: proportion, dtype: float64

### Cenário 8: Excluir todos os clientes do plano mensal e com idade acima de 50 anos

In [51]:
## excluindo os clientes do plano mensal e com idade acima de 50 anos
tabela_filtro_assinatura_e_idade = tabela.loc[(tabela['duracao_contrato'] !='Monthly') & (tabela['idade'] <= 50),:]
tabela_filtro_assinatura_e_idade['cancelou'].value_counts()
tabela_filtro_assinatura_e_idade['cancelou'].value_counts(normalize=True)

cancelou
0.0    0.637006
1.0    0.362994
Name: proportion, dtype: float64

## 4. Análise Multivariáveis (Churn vs Outras Variáveis)
- Compare as características dos clientes que cancelaram (cancelou = 1) com os que não cancelaram (cancelou = 0).
- Idade média dos que cancelam vs. os que permanecem.
- Relação entre total gasto e cancelamento.
- Clientes que mais ligam para o call center têm maior taxa de churn?
- Clientes com mais tempo de contrato têm menor chance de cancelar?

#### Cenário 9: Excluir os clientes do plano mensal e que atrasaram o pagamento mais de 20 dias e que ligaram mais de 5 vezes 

In [52]:
## excluindo os clientes do plano mensal e que atrasaram o pagamento mais de 20 dias e que ligaram mais de 5 vezes 
tabela_filtro_assinatura_e_dias_atraso_ligacoes = tabela.loc[(tabela['duracao_contrato'] !='Monthly') & (tabela['dias_atraso'] <= 20) & (tabela['ligacoes_callcenter'] <= 5) ,:]
tabela_filtro_assinatura_e_dias_atraso_ligacoes['cancelou'].value_counts()
tabela_filtro_assinatura_e_dias_atraso_ligacoes['cancelou'].value_counts(normalize=True)

cancelou
0.0    0.781358
1.0    0.218642
Name: proportion, dtype: float64

## 5. Insights e Recomendações
- Identificamos que os clientes cancelaram seus planos de acordo com as situações abaixo:
    - Todos os clientes do plano Mensal cancelaram
    - Clientes com a partir de 51 anos, cancelaram
    - Clientes que realizaram, a partir de 5 ligações para o Call Center, cancelaram
    - Clientes que tiveram a partir de 21 dias de atraso, cancelaram
    
- Taxas de cancelamento:
    - Taxa de cancelamento = 56,7 %
    - Taxa de cancelamento excluindo os clientes do plano mensal = 46 %
    - Taxa de cancelamento excluindo todos os clientes que ligaram mais de 5 vezes = 40,8 %
    - Taxa de cancelamento excluindo os clientes que atrasaram o pagamento mais de 20 dias ou com idade acima de 50 anos = 46 %
    - Taxa de cancelamento excluindo todos os clientes do plano mensal e que ligaram mais de 5 vezes = 30,5 %
    - Taxa de cancelamento excluindo os clientes do plano mensal, os que atrasaram o pagamento mais de 20 dias e que ligaram mais de 5 vezes = 21,8 %

- Recomendação para redução do Churn:
    - Condições melhores para os planos Mensais ou exclusão desses
    - Redução da necessidade de muitas ligações para o Call Center
    - Acompanhamento próximo das pessoas que atrasarem o pagamento
